In [0]:
'''
Definindo funções da camada Silver a serem utilizadas entre notebooks
'''

def clean_and_cast_to_int(df, column_name):
    # Remover todos os caracteres não numéricos
    df = df.withColumn(column_name, F.regexp_replace(df[column_name], '\D', ''))
    
    # Transformar valores não nulos em inteiros
    df = df.withColumn(
        column_name,
        F.when(F.col(column_name).isNotNull() & (F.col(column_name) != ''), F.col(column_name).cast("int"))
         .otherwise(None)
    )
    return df

# Funções de Teste
def test_col_not_null(df, col_name):
    print(f"Avaliando a condição {col_name} não contem nulos")
    # Filtra linhas onde a coluna é nula
    null_count = df.filter(F.col(col_name).isNull()).count()
    
    # Verifica se o número de valores nulos é zero
    assert null_count == 0, f"Coluna {col_name} contém valores null"

def test_biggest_date_before_current_date(df, col_name):
    print(f"Avaliando a condição {col_name} menor que a data atual")
    # Obter a data atual
    current_date = F.current_date()

    # Obter a maior data na coluna especificada
    max_date = df.agg(F.max(col_name)).collect()[0][0]
    
    # Verificar se a maior data é antes da data atual
    assert max_date <= df.select(current_date).collect()[0][0], f"A maior data na coluna {col_name} é posterior a data atual"


def test_value_range(df, col_name, condition):
    df_filtered = df.filter(F.expr(f"{col_name} {condition}"))
    print(f"Avaliando a condição {col_name} {condition}")
    #df_filtered.display()
    assert df.count() == df_filtered.count(), f"Os dados da coluna {col_name} não atendem a condição {condition}"


def test_weekday(df, col_name):
    print(f"Avaliando a condição {col_name} está dentre os valores aceitos para o dia de semana")
    # Obter os valores distintos da coluna
    distinct_values = df.select(col_name).distinct().collect()
    
    # Converter os valores distintos para uma lista de strings
    distinct_values_list = [row[col_name] for row in distinct_values]
    
    # Lista de dias da semana
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    
    # Verificar se todos os valores distintos estão na lista de dias da semana
    for value in distinct_values_list:
        assert value in weekdays, f"Os dados da coluna {col_name} não correspondem a dias de semana: {value}"

def test_IATA_codes(df, col_name):
    print(f"Avaliando a condição {col_name} contem 3 caracteres")
    nb_different = df.filter(F.length(F.col(col_name)) != 3).count()
    assert nb_different == 0, "Existem códigos IATA além de 3 dígitos"

def test_ICAO_codes(df, col_name):
    print(f"Avaliando a condição {col_name} contem 4 caracteres")
    nb_different = df.filter(F.length(F.col(col_name)) != 4).count()
    assert nb_different == 0, "Existem códigos IATA além de 4 dígitos"